<a target="_blank" href="https://colab.research.google.com/github/DeepLabCut/DeepLabCut/blob/main/examples/COLAB/COLAB_DEMO_SuperAnimal.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# First, go to "Runtime" ->"change runtime type"->select "Python3", and then select "GPU"

In [ ]:
!pip install "deeplabcut[tf,modelzoo] @ git+https://github.com/DeepLabCut/DeepLabCut@main"

## PLEASE, click "restart runtime" from the output above before proceeding! 

In [5]:
import deeplabcut
import os

In [ ]:
from google.colab import files

uploaded = files.upload()
for filepath, content in uploaded.items():
  print(f'User uploaded file "{filepath}" with length {len(content)} bytes')
video_path = os.path.abspath(filepath)

# If this cell fails (e.g., when using Safari in place of Google Chrome),
# manually upload your video via the Files menu to the left 
# and define `video_path` yourself with right click > copy path on the video.

In [6]:
supermodel_name = "superanimal_topviewmouse" #@param ["superanimal_topviewmouse", "superanimal_quadruped"]
pcutoff = 0.3 #@param {type:"slider", min:0, max:1, step:0.05}

In [ ]:
videotype = os.path.splitext(video_path)[1]
scale_list = []
deeplabcut.video_inference_superanimal(
    [video_path],
    supermodel_name,
    videotype=videotype,
    video_adapt=True,
    scale_list=scale_list,
    pcutoff=pcutoff,
)